# Flask-based resume screening application looks well-structured! Here are some suggestions and improvements:

In [3]:
pip install docx2txt


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3973 sha256=6538047f9ba7e172edb60c3ca59c088a34c799cc3f866db0e42f8531a268de0b
  Stored in directory: c:\users\shrin\appdata\local\pip\cache\wheels\6f\81\48\001bbc0109c15e18c009eee300022f42d1e070e54f1d00b218
Successfully built docx2txt


In [6]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [12]:
from docx import Document

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])


In [30]:
import os
import re
import docx2txt
import PyPDF2
import spacy
import pandas as pd
from flask import Flask, request, render_template
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load NLP model
nlp = spacy.load('en_core_web_sm')

app = Flask(__name__)

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + " "
    return text

def extract_text_from_docx(docx_path):
    return docx2txt.process(docx_path)

def extract_text_from_resume(file_path):
    if file_path.endswith('.pdf'):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith('.docx'):
        return extract_text_from_docx(file_path)
    return ""

def extract_skills(text):
    skills = set()
    common_skills = {"Python", "Java", "C++", "Machine Learning", "AI", "Data Science", "Flask", "Django"}
    doc = nlp(text)
    for token in doc:
        if token.text in common_skills:
            skills.add(token.text)
    return list(skills)

def extract_experience(text):
    experience = re.findall(r'\d+\s*(?:years|months)', text, re.IGNORECASE)
    return ", ".join(experience)

def calculate_similarity(resume_text, job_desc):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([resume_text, job_desc])
    return cosine_similarity(vectors[0], vectors[1])[0][0]

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        job_desc = request.form['job_desc']
        uploaded_files = request.files.getlist("resumes")
        results = []
        for file in uploaded_files:
            file_path = os.path.join("uploads", file.filename)
            file.save(file_path)
            text = extract_text_from_resume(file_path)
            skills = extract_skills(text)
            experience = extract_experience(text)
            similarity_score = calculate_similarity(text, job_desc)
            results.append({
                "filename": file.filename,
                "skills": skills,
                "experience": experience,
                "score": round(similarity_score * 100, 2)
            })
        results = sorted(results, key=lambda x: x['score'], reverse=True)
        return render_template('index.html', results=results)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\shrin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install spacy

   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
    --------------------------------------- 0.3/11.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.8 MB 1.5 MB/s eta 0:00:08
   -- ------------------------------------- 0.8/11.8 MB 1.3 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/11.8 MB 1.4 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/11.8 MB 1.6 MB/s eta 0:00:07
   ------- -------------------------------- 2.1/11.8 MB 1.8 MB/s eta 0:00:06
   -------- ------------------------------- 2.6/11.8 MB 1.9 MB/s eta 0:00:05
   ---------- ----------------------------- 3.1/11.8 MB 2.0 MB/s eta 0:00:05
   ------------ --------------------------- 3.7/11.8 MB 2.1 MB/s eta 0:00:04
   --------------- ------------------------ 4.5/11.8 MB 2.2 MB/s eta 0:00:04
   ----------------- ---------------------- 5.2/11.8 MB 2.3 MB/s eta 0:00:03
   -------------------- ------------------- 6.0/11.8 MB 2.5 MB/s eta 0:00:03
   ----------

In [27]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.7 MB/s eta 0:00:08
     ---- ----------------------------------- 1.3/12.8 MB 2.6 MB/s eta 0:00:05
     ----- ---------------------------------- 1.8/12.8 MB 2.6 MB/s eta 0:00:05
     ------- -------------------------------- 2.4/12.8 MB 2.7 MB/s eta 0:00:04
     --------- ------------------------------ 3.1/12.8 MB 3.0 MB/s eta 0:00:04
     ------------- -------------------------- 4.2/12.8 MB 3.2 MB/s eta 0:00:03
     --------------- ------------------------ 5.0/12.8 MB 3.4 MB/s eta 0:00:03
     ------------------ --------------------- 6.0/12.8 MB 3.5 MB/s eta 0:00:02
     --------------------- ------------------ 6.8/12.8 MB 3.6 MB/s eta 0:00:02
     ------------------------ --------------- 7.9/12.8 MB 3.6 MB/s eta 0:00:02
     --------------------------- ------------ 8.7/12.8 MB 3.7 MB/s

# Write a program that takes a list of numbers as input and returns the largest number in the list.

In [33]:
def find_max(num):
    if not num:
        return None
    return max(num)

num = list(malargest = find_largest_number(numbers)
largest = find_max(numbers)

if largest is not None:
    print("The largest number is:", largest)
else:
    print("The list is empty.")

SyntaxError: '(' was never closed (26895614.py, line 6)